# Designing a Survey 

Within this notebook i am going to briefly demonstrate how you, the user, can design a simple Roman survey, making some assumptions. 

In [1]:
import matplotlib
import numpy as np
import os, sys, glob
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from scipy.interpolate import interp1d
from scipy import interp
import astropy.io
from astropy.io import fits
from astropy.io import ascii
import math

# Instrument and observatory inputs. 

To define our survey we need to think about what instruments we want to use and how. 
For this tutorial we will creating a two tier imaging survey using filters from the Wide Field Instrument.

## Filters

The Roman WFI has eight imgaing filters, one prism, and one grism. The zero-points of the imaging filters and prism are specified below. **Jeff can you check these please**

In [2]:
ZP_R = 26.56
ZP_Z = 26.41
ZP_Y = 26.44
ZP_J = 26.40
ZP_H = 26.43
ZP_F = 25.95
ZP_W = 27.65
ZP_Prism = 27.70

ZPs = {'R':float(ZP_R), 'Z':float(ZP_Z), 'Y':float(ZP_Y), 'J':float(ZP_J), 'H':float(ZP_H), 'F':float(ZP_F), 'W':float(ZP_W)}

# Zodical light
These are the zodical light measurements taken from WFIRS_WFI_Reference_Information_20190628

They are count rate per pixel at minimum zodi
(e-/s/pix)

For observations at high galactic latitudes, the Zodi intensity is typically ~1.5x the minimum. For observation into the galactic bulge the Zodi intensity is typically 2.5-7x the minimum.

In [3]:
zod_R = 0.27
zod_Z = 0.27
zod_Y = 0.29
zod_J = 0.28
zod_H = 0.26
zod_F = 0.15
zod_W = 0.85
zod_K = 0.13

zodi = {'R':float(zod_R),'Z':float(zod_Z),'Y':float(zod_Y),'J':float(zod_J),'H':float(zod_H),'F':float(zod_F),'W':float(zod_W), 'K':float(zod_K)}

# Thermal Background 
Estimated thermal background e-/s

In [1]:
ther_R = 0.0
ther_Z = 0.0
ther_Y = 0.0
ther_J = 0.0
ther_H = 0.04
ther_F = 0.17
ther_W = 0.98
ther_K = 4.52

ther = {'R':float(ther_R),'Z':float(ther_Z),'Y':float(ther_Y),'J':float(ther_J),'H':float(ther_H),'F':float(ther_F),'W':float(ther_W),'K':float(ther_K)}

# Dark Current
This is expected to be less than 0.005 e-/pix/sec

In [5]:
dark = 0.005

In [6]:
def read_noise_calc(exp):
    RD = 2.825
    read_noise = np.sqrt((5**2)+12*(20**2)*(exp/RD-1)/(exp/RD)/(exp/RD+1))

    return read_noise

# Readnoise

The read-out time is 2.825 seconds

We calcualte the read noise based on exposure times and an equation adjusted from Rauscher et al. 2007

In [7]:
def background_noise(zod,ther,dark,exp):
    sky_sig = np.sqrt((zod*exp)+(ther*exp)+(dark*exp))
    
    return sky_sig

# PSF FWHM Values

Values are quoted in arcseconds. Note that the PSF FWHM in arcseconds are simulated at the center of the center pixel of a detector near the center of the WFI FOV using an input spectrum for a K0V type star. 

In [8]:
PSF_R = 0.058
PSF_Z = 0.073
PSF_Y = 0.087
PSF_J = 0.105
PSF_H = 0.127
PSF_F = 0.151
PSF_W = 0.105
PSF_K = 0.175

PSFs = {'R':float(PSF_R),'Z':float(PSF_Z),'Y':float(PSF_Y),'J':float(PSF_J),'H':float(PSF_H),'F':float(PSF_F), 'W':float(PSF_W), 'K':float(PSF_K)}

# Areas

For each tier you are going to want to assign different areas.
A single Roman ponting covers 0.281 square degrees.

In [9]:
pointing = 0.281    

In [10]:
def area_calc(area):
    field = float(0.281) #size of each pointing
    pt = area/field
    pointings = math.trunc(pt)
    
    area_deg = pointings * field
    
    return area_deg, pointings

# Exposure Times

You are probably going to want to taylor your exosure time based on the S/N you require for your kind of objects. 

You can use the ETC calculator provided [here](add link to ETC calculator) to do this. You can the plan your survey sized based on how many object you expect in a given area, and how long you want to observe that area to get the requred S/N.

For this tutorial I am focusing on the creating of a two tier supernova survey. The shallow part of the survey will focus on obtaining SNe in the R, Z, Y, J bands, ensuring a S/N of at least 10 for a magnitude of 25. Using the ETC, I get the following exposure times (when the selected zodical light contribution is 2).

- R = 73.0 s
- Z = 110.0 s
- Y = 110.0 s
- J = 119.0 s

Note that its probably not wise to have an exposure time less than 100s as you will dominated by readout noise. As such we will change the R-band exposure time to 100.

The cadence of my observation in this shallow field will 5 days.

For the deep field i want to target SNe at a magnitude of 26. I will select the YJHF filters and have a cadence of 5 days. 

- Y = 393.0 s
- J = 428.0 s
- H = 484.0 s
- F = 1077.0 s

In [11]:
##SHALLOW###
tier1_exp = [100.0, 110.0, 110.0, 119.0]
cadence_shallow = 5

##DEEP###
tier2_exp = [393.0, 428.0, 484.0, 1077.0]
cadence_deep = 5

In [16]:
##TOTAL VALUES###
Shallow_tot = sum(tier1_exp)
Deep_tot = sum(tier2_exp)

print(Shallow_tot, Deep_tot)

439.0 2382.0


From the [Slew Times table](link that page) we note that the time for a shift of the entire detector by one pointing is roughly 70 seconds. As such we will use this as our estimated overhead. 

In [13]:
overhead = 70

You can use the information in the [slew table](add link here to the table) to do a more detailed analysis, but for now this is what we will use.

# Survey Creation

We now need to take each of these componants and create a survey.

We know the total amount of time avalible is 700 hrs - so one month.
This can be conducted over the first two years.

The total exposure time in the shallow field is 439 seconds, and 2382 seconds for the deep.
**However** we still need to include our overhead which is 70 seconds per filter, assimung we will be imaging our survey areas all in one fiter, switch, and re-observe with the next.

The final shallow survey time per pointing is then 439+(4 x 70) = 719 s, and 2382+(4 x 70) = 2662 s for the deep. 

The total time is then 3381 seconds, so just under an hr.


Since the cadence is 5 days, and the survey can be conducted over two years we know that there should be 146 visits.

We can also play around with the length of each visit.

In [14]:
# Shallow
exp1= Shallow_tot

# Deep
exp2 = Deep_tot

#Telescope info
slew = 70 #sec
cadence = 5
length = 2*365 #Since spread over two years
num_visits = length/cadence #Since we are 
Tot_time = 29.17 #days

#You specify what filters you want
tier1_flt = ['R','Z','Y','J'] 
tier2_flt = ['Y','J','H','F']

# Get the times for the 1st tier
tot_time1 = sum(tier1_exp)+(slew*len(tier1_flt))
   
# Get the times for the 2nd tier
tot_time2 =  sum(tier2_exp)+(slew*len(tier2_flt))
    
#Now list the tiers you want
print("Tier 1")
print("Filters:", tier1_flt)
print("Exp times:", tier1_exp)
print("Total Time:", tot_time1)
print("\n")

print("Tier 2")
print("Filters:", tier2_flt)
print("Exp times:", tier2_exp)
print("Total Time:",tot_time2)
print("\n")

Tier 1
Filters: ['R', 'Z', 'Y', 'J']
Exp times: [100.0, 110.0, 110.0, 119.0]
Total Time: 719.0


Tier 2
Filters: ['Y', 'J', 'H', 'F']
Exp times: [393.0, 428.0, 484.0, 1077.0]
Total Time: 2662.0




In [17]:
#Now figure out the areas

tot_time1_hrs = tot_time1/(60*60)
tot_time2_hrs = tot_time2/(60*60)

# You need to input rough areas
DA = 1
SA = 4

DA2, DP = area_calc(DA)
SA2, SP = area_calc(SA)
Areas = {'SHALLOW': SA2, 'DEEP': DA2}
print("Areas (square degrees):", Areas)

ST = tot_time1_hrs * SP
DT =tot_time2_hrs * DP

t_visit = (ST+DT)
t_survey = ((ST+DT)*num_visits)/24

print("Time per visut (hrs): Shallow = ", ST, "Deep =",DT, "\n Total Visit time (hrs) = ", t_visit, "\n Survey Time (days) = ", t_survey)


Areas (square degrees): {'SHALLOW': 3.934, 'DEEP': 0.8430000000000001}
Time per visut (hrs): Shallow =  2.796111111111111 Deep = 2.2183333333333333 
 Total Visit time (hrs) =  5.014444444444445 
 Survey Time (days) =  30.50453703703704


Ok we have figured out the areas for the two given our timing constraint. Note that the total time is just over that alloted, and we may need to adjust for that. However, this gives you the user the basic guidelines for establishing your surveys.